### Introduction to Data Ingestion and Parsing


In [2]:
import os
from typing import List, Dict, Any
import pandas


In [4]:
from langchain_core.documents import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
)
print("Set up is complete!")


Set up is complete!


### Understanding the Document Structure in LangChain

In [5]:
## Create a simple document
doc = Document(
    page_content="This is the main text content that will be embedded and searched.",
    metadata={
        "source": "example.txt",
        "page": 1,
        "author": "Johnny Boy",
        "date_created": "2025-09-02",
        "custom_field": "custom_value",
    },
)
print("Document structure");
print(f"Content: {doc.page_content}");
print(f"Metadata: {doc.metadata}");

Document structure
Content: This is the main text content that will be embedded and searched.
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Johnny Boy', 'date_created': '2025-09-02', 'custom_field': 'custom_value'}


### Text Files (.txt) - The simplest case {#2-text-files}

In [2]:
## Create a simple text file
import os
os.makedirs("data/text_files", exist_ok=True)


In [5]:
sample_texts = {
    "data/text_files/python_intro.txt": """1-dataingestion.ipynb
Python's dynamic typing allows flexible code but can cause runtime errors.
The GIL prevents true multithreading, making multiprocessing better for CPU tasks.
List comprehensions replace for loops with readable one-liners.
Python emphasizes readability and the principle of one obvious way.
The standard library is extensive, earning Python the 'batteries included' nickname.
Duck typing determines suitability by methods, not actual type.
Python's interpreter enables rapid prototyping and interactive development.
Data science popularity comes from NumPy, Pandas, and Scikit-learn libraries.
Python's natural language syntax makes it ideal for beginners.
Virtual environments solve dependency conflicts between projects
    """,
    "data/text_files/machine_learning.txt": """
Machine learning algorithms learn patterns from data without explicit programming.
Supervised learning uses labeled data to predict outcomes on new examples.
Unsupervised learning finds hidden structures in data without target labels.
Neural networks mimic brain neurons with interconnected layers of nodes.
Deep learning uses multiple hidden layers to model complex relationships.
Overfitting occurs when models memorize training data but fail on new data.
Cross-validation splits data to test model performance on unseen examples.
Feature engineering transforms raw data into meaningful model inputs.
Gradient descent optimizes model parameters by minimizing prediction errors.
Ensemble methods combine multiple models to improve overall accuracy.
    """
}

for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"{filepath} created")


data/text_files/python_intro.txt created
data/text_files/machine_learning.txt created
